This script automates the process of creating a searchable vector database from the Godot game engine's documentation.

In [ ]:
# --- 1. Install all dependencies ---
!pip install -U langchain langchain-community langchain-huggingface sentence-transformers chromadb docutils

import os
import shutil
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from docutils.core import publish_parts
from tqdm.auto import tqdm
from google.colab import drive

# --- 2. Clone the repository ---
repo_url = "https://github.com/godotengine/godot-docs.git"
repo_path = "godot-docs-4.4"
branch = "4.4"

if not os.path.exists(repo_path):
    print(f"Cloning repository from {repo_url} on branch {branch}...")
    !git clone --branch {branch} {repo_url} {repo_path} --progress
else:
    print(f"Directory '{repo_path}' already exists. Skipping clone.")

# --- 3. Find and convert .rst files ---
rst_files = []
for root, dirs, files in os.walk(repo_path):
    for file in files:
        if file.endswith('.rst'):
            rst_files.append(os.path.join(root, file))

print(f"Found {len(rst_files)} .rst files.")

def rst_to_string(file_path):
    """Converts an .rst file to a plain text string."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            rst_content = f.read()
        parts = publish_parts(source=rst_content, writer_name='html4css1', settings_overrides={'halt_level': 5, 'report_level': 5})
        return parts['body']
    except Exception as e:
        return ""

text_file = "godot_docs.txt"
if not os.path.exists(text_file):
    print(f"Converting .rst files to {text_file}...")
    all_text = []
    for rst_file in tqdm(rst_files, desc="Converting .rst files"):
        all_text.append(rst_to_string(rst_file))
    full_text = "\n".join(all_text)
    with open(text_file, "w", encoding="utf-8") as f:
        f.write(full_text)
    print(f"Saved the combined text to {text_file}")
else:
    print(f"File '{text_file}' already exists. Skipping conversion.")
    with open(text_file, "r", encoding="utf-8") as f:
        full_text = f.read()

# --- 4. Chunk and embed the data ---
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(full_text)
print(f"Created {len(chunks)} chunks.")

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# --- 5. Store in ChromaDB ---
db_path = "./chroma_db"
if os.path.exists(db_path):
    shutil.rmtree(db_path)

if chunks:
    print("Creating vector database. This may take a while...")
    vectordb = Chroma.from_texts(
        texts=chunks,
        embedding=embeddings,
        persist_directory=db_path
    )
    print("Vector database created successfully.")
else:
    print("No text chunks to process. Vector database not created.")

# --- 6. Save to Google Drive ---
drive.mount('/content/drive')
drive_db_path = '/content/drive/My Drive/chroma_db'
if os.path.exists(drive_db_path):
    shutil.rmtree(drive_db_path)
shutil.copytree(db_path, drive_db_path)
print(f"Successfully saved the vector database to your Google Drive at {drive_db_path}")

# Godot Engine 4.4 Documentation Vector Database

This project provides a pre-built vector database for the Godot Engine 4.4 documentation. This database can be used to perform fast and efficient similarity searches on the entire Godot documentation, making it a powerful tool for developers looking for information, code examples, or tutorials.

## What is a Vector Database?

A vector database is a type of database that stores and indexes data as high-dimensional vectors. In the context of natural language processing, these vectors represent the semantic meaning of text. By converting the Godot documentation into a vector database, we can perform searches based on the meaning of the query, rather than just keywords. This allows for more intuitive and powerful searches. For example, you could search for "how to make a character jump" and get relevant results even if the exact phrase doesn't appear in the documentation.

## How was this database created?

This vector database was created using the following process:

1. **Cloning the Documentation:** The official Godot Engine 4.4 documentation was cloned from its GitHub repository.
2. **Parsing .rst Files:** All of the `.rst` (reStructuredText) files in the documentation were parsed and converted into plain text.
3. **Chunking:** The entire text of the documentation was split into smaller chunks of 1000 characters with a 200-character overlap. This is done to ensure that the context of each chunk is preserved.
4. **Embedding:** Each chunk of text was then converted into a high-dimensional vector using the `all-MiniLM-L6-v2` sentence transformer model. This model is a state-of-the-art model that is optimized for creating sentence and paragraph embeddings.
5. **Storing in ChromaDB:** The resulting vectors were then stored in a ChromaDB database. ChromaDB is an open-source vector database that is easy to use and can be run locally.

## How to use this database

To use this database, you will need to have Python installed, along with the following libraries:

* `langchain`
* `langchain-community`
* `langchain-huggingface`
* `sentence-transformers`
* `chromadb`

You can install these libraries using pip:

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Load the vector database from your Google Drive
drive_db_path = '/content/drive/My Drive/chroma_db'
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
vectordb = Chroma(persist_directory=drive_db_path, embedding_function=embeddings)

# Perform a similarity search
query = "how to create a 2d character animation"
docs = vectordb.similarity_search(query)

# Print the results
for doc in docs:
    print(doc.page_content)
    print("-" * 20)

This will perform a similarity search for the query "how to create a 2d character animation" and print the most relevant documents from the Godot documentation.

## Conclusion

This vector database provides a powerful and intuitive way to search the Godot Engine 4.4 documentation. By leveraging the power of vector embeddings, you can find the information you need quickly and easily. We hope that this project will be a valuable resource for the Godot community.